# Creating a project from an existing dataset
In this notebook we'll use the `geti-sdk` package to create a project from an existing dataset, and upload images and annotations to it.

### Setting up the connection to the platform
First, we set up the connection to the cluster. This is done by instantiating a Geti object, with the hostname (or ip address) and login details for the cluster. As in notebook [001 create_project](001_create_project.ipynb), the login details are stored in the `.env` file and are loaded using the `dotenv` package.

In [ ]:
from dotenv import dotenv_values

env_variables = dotenv_values(dotenv_path=".env")

if not env_variables:
    print(
        "Unable to load login details from .env file, please make sure the file exists at the root of the notebooks directory."
    )

In [ ]:
from geti_sdk import Geti

geti = Geti(
    host=env_variables.get("HOST"),
    username=env_variables.get("USERNAME"),
    password=env_variables.get("PASSWORD"),
)

### Getting the COCO dataset
In the next cell, we get the path to the MS COCO dataset. 

If you already have the COCO dataset on your machine, please specify the `dataset_path` to point to the folder containing the dataset. 

If you don't have the dataset yet, the `get_coco_dataset` method will make an attempt to download the dataset. Even though it will only download the 2017 validation subset, this is still a ~1 Gb download so it may take some time, depending on your internet connection. 

Of course the data will only be downloaded once; if you have downloaded the dataset previously, the method should detect it and return the path to the data.

In [ ]:
from geti_sdk.utils import get_coco_dataset

COCO_PATH = get_coco_dataset(dataset_path=None)

### Reading the dataset
Next, we need to load the COCO dataset using Datumaro. The `geti-sdk` package provides the `DatumAnnotationReader` class to do so. It can read datasets in all formats supported by Datumaro.

In [ ]:
from geti_sdk.annotation_readers import DatumAnnotationReader

annotation_reader = DatumAnnotationReader(
    base_data_folder=COCO_PATH, annotation_format="coco"
)

### Selecting the labels
The MS COCO dataset contains 80 different classes, and while we could create a project including all of them, for this demo we'll select only a couple of them. This is done using the `filter_dataset` method of the annotation reader.

In [ ]:
annotation_reader.filter_dataset(labels=["dog", "cat", "horse"], criterion="OR")

### Creating the project
Now that we have a selection of data we'd like to upload, we get to create the project. The COCO dataset is best suited for detection or segmentation type projects. 

To create the project, we'll be using a method `create_single_task_project_from_dataset` from the `Geti` instance that we set up previously. This will not only create the project, but also upload the media and annotations from our dataset. 

The project name and type can be set via their respective input parameters, `project_name` and `project_type`. Have a look at notebook [001 create project](./001_create_project.ipynb) for further details about which values are supported for the `project_type` parameter.

The number of images that is uploaded and annotated can be controlled as well. Finally, if `enable_auto_train` is set to `True` the project will start training right after all annotations have been uploaded (provided that sufficient images have been annotated to trigger auto-training).

In [ ]:
PROJECT_NAME = "COCO animal detection demo"
PROJECT_TYPE = "detection"

In [ ]:
project = geti.create_single_task_project_from_dataset(
    project_name=PROJECT_NAME,
    project_type=PROJECT_TYPE,
    path_to_images=COCO_PATH,
    annotation_reader=annotation_reader,
    number_of_images_to_upload=100,
    number_of_images_to_annotate=90,
    enable_auto_train=True,
)

That's it! A new project named `COCO animal detection demo` should now appear in your workspace. To check it's properties, we can print a summary of it in the cell below.

In [ ]:
print(project.summary)

As you probably notice there is one additional label in the project, the `No Object` label. This is added by the system automatically to represent the absence of any 'horse', 'cat' or 'dog' in an image.